In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import scipy.sparse as sps
from itertools import combinations
import numpy as np

In [2]:
data_manager=MovieLens100kDataManager()

In [3]:
# print(dataset.load_rating_predefined_split(3))
ratings = data_manager.load_rating_all().drop(columns=['timestamp'])
users = data_manager.load_user_info()
columns_users = []
for col in users.columns:
    if str(col) != 'user_id' :
        columns_users.append('user_'+str(col))
    else:
        columns_users.append(str(col))
users = pd.DataFrame(users.values,columns=columns_users)
#Merge tập ratings và users được gom nhóm bởi user_id
result =  pd.merge(ratings, users, on="user_id", how="outer")

movies = data_manager.load_movie_info()
columns_movies = []
for col in movies[['movie_id','release_date','genres']].columns:
    if str(col) != 'movie_id' :
        columns_movies.append('movie_'+str(col))
    else:
        columns_movies.append(str(col))
movies = pd.DataFrame(movies[['movie_id','release_date','genres']].values,columns=columns_movies)
result=  pd.merge(result, movies, on="movie_id", how="outer")

In [4]:
result

,user_id,movie_id,rating,user_age,user_gender,user_occupation,user_zipcode,movie_release_date,movie_genres
0,196,242,3,49,M,writer,55105,1997-01-24,Comedy
1,305,242,5,23,M,programmer,94086,1997-01-24,Comedy
2,6,242,4,42,M,executive,98101,1997-01-24,Comedy
3,234,242,4,60,M,retired,94702,1997-01-24,Comedy
4,63,242,3,31,M,marketing,75240,1997-01-24,Comedy
...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,17,M,student,60089,1998-02-06,Romance|Thriller
99996,863,1678,1,17,M,student,60089,1998-02-06,Drama
99997,863,1680,2,17,M,student,60089,1998-01-01,Drama|Romance
99998,896,1681,3,28,M,writer,91505,1994-01-01,Comedy


In [10]:
def get_user_info(result):
    user_feature_name = [col for col in result.columns if 'user' in col]
    user_data = result[user_feature_name].drop_duplicates(subset = "user_id").set_index('user_id').sort_values(by='user_id')
    return user_data

In [11]:
get_user_info(result)

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,user_age,user_gender,user_occupation,user_zipcode
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [12]:
def get_movie_info(result):
    movie_feature_name = [col for col in result.columns if 'movie' in col]
    movie_data = result[movie_feature_name].drop_duplicates(subset = "movie_id").set_index('movie_id').sort_values(by='movie_id')
    return movie_data

In [13]:
get_movie_info(result)

c:\Python310\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,movie_release_date,movie_genres
movie_id,,
1,1995-01-01,Animation|Children's|Comedy
2,1995-01-01,Action|Adventure|Thriller
3,1995-01-01,Thriller
4,1995-01-01,Action|Comedy|Drama
5,1995-01-01,Crime|Drama|Thriller
...,...,...
1678,1998-02-06,Drama
1679,1998-02-06,Romance|Thriller
1680,1998-01-01,Drama|Romance
